In [1]:
import sys
sys.path.append(r"D:\Scripts1\Code\ActPEA\CODE\Actu_Colums.py")
import pandas as pd
import numpy as np
import Actu_Colums as Ac
import win32com.client as win32
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
import datetime
import os
import glob
from typing import Optional
from babel.dates import format_date, format_datetime, Locale
import re
import pickle



### El flujo de actualizacion es el siguiente : 
    -01.Extraigo un fragmente del prepa_global_N solo de EI
    -01,5. Concateno los Bloqueos de Factura
    -02.Hereda informacion del Prepa_old
    -03.Actualizo info del PAP
    -03,5. Añado analistas
    -04.Vuelvo a meter el fragmento en el prepa_global_N
    -05.To Excel

## Time pointer and funcions

In [2]:
Today_str = datetime.date.today().strftime('%d-%m-%Y')
Today_D_M = Today_str[0:5]
Today_D_M = [Today_D_M[0:2],'.',Today_D_M[3:6]]
Today_D_M = ''.join(Today_D_M)
#Today_D_M = '06.02'
print('Hoy es:',Today_D_M)
def process_df_V1(df: pd.DataFrame,clases_unicas: list,index=['NOMPROVEEDOR','ESTADO']):
    """ La funcion recibe un df, y las clases unicas
        Y devuelve el df uniformizado para cada clase
        Llena los valores faltantes con 0"""
    pivot_table = pd.pivot_table(df, values='PEND_FACT_SOLES', index=index, aggfunc=pd.Series.sum) #Agrupo 
    grupos = pivot_table.groupby('NOMPROVEEDOR') #creo un data frame para cada contrata
    # Para cada grupo, crea un nuevo dataframe y guárdalo en un diccionario
    dataframes = {}
    for nombre, datos in grupos: # Creo los df dentro del dic
        dataframes[nombre] = datos
    for key in dataframes.keys(): # para cada df en el dic
        df = dataframes[key].reset_index()
        df1 = df.set_index('ESTADO').reindex(clases_unicas).reset_index() #normalizo, creando filas para todos los ESTADOS
        df1.NOMPROVEEDOR = key  #Relleno los NAN
        df1 = df1.fillna(0)  
        dataframes[key] = df1   #Reescribo los Df para cada clave
    df_concat = pd.concat(dataframes.values(),ignore_index=True) # Compacto todo los df del dic en uno grande 
    pivot_table_1 = pd.pivot_table(df_concat, values='PEND_FACT_SOLES', index=index, aggfunc=pd.Series.sum) #agrupo de nuevo
    return pivot_table_1
################################################################################################3
def process_df_V2(df: pd.DataFrame,clases_unicas: list,index=['NOMPROVEEDOR','ESTADO'],agg = pd.Series.sum):
    """ La funcion recibe un df, y las clases unicas
        Y devuelve el df uniformizado para cada clase
        Llena los valores faltantes con 0"""
    pivot_table = pd.pivot_table(df, values='PEND_FACT_SOLES', index=index, aggfunc=agg) #Agrupo 
    grupos = pivot_table.groupby(index[:-1]) #creo un data frame para cada index mayor
    # Para cada grupo, crea un nuevo dataframe y guárdalo en un diccionario
    dataframes = {}
    for nombre, datos in grupos: # Creo los df dentro del dic
        dataframes[nombre] = datos
    for key in dataframes.keys(): # para cada df en el dic
        df = dataframes[key].reset_index()
        df1 = df.set_index(index[-1]).reindex(clases_unicas).reset_index() #normalizo, creando filas para todos los ESTADOS
        df1[index[:-1]] = key  #Relleno los NAN para normalizar estados
        df1 = df1.fillna(0)  
        dataframes[key] = df1   #Reescribo los Df para cada clave
    df_concat = pd.concat(dataframes.values(),ignore_index=True) # Compacto todo los df del dic en uno grande 
    pivot_table_1 = pd.pivot_table(df_concat, values='PEND_FACT_SOLES', index=index, aggfunc=pd.Series.sum) #agrupo de nuevo
    return pivot_table_1
def calc_diff_V2(df1: pd.DataFrame,df2: pd.DataFrame, index= ['NOMPROVEEDOR', 'ESTADO'],OutC = 'DIFERENCIA EN SOLES',agg = pd.Series.sum) -> pd.DataFrame:
    """ entran 2 dataframes , los proceso , y en base a los df procesados 
        calculo la diferencia en una nueva columna, 
        y añado esta columna de diferencia al df actual """
    df1 = cleanrows(df1)
    df2 = cleanrows(df2)
    clases_unicas = pd.concat([df1['ESTADO'].dropna(), df2['ESTADO'].dropna()]).unique() #Creo listas de claves unicas
    df1_proces = process_df_V2(df1,clases_unicas)
    df2_proces = process_df_V2(df2,clases_unicas)
    
    df_diff = df1_proces - df2_proces
    diff_pivot_table_reset = df_diff.reset_index()
    diff_pivot_table_reset.rename(columns={'PEND_FACT_SOLES': OutC}, inplace=True)
    # Merge diff_pivot_table_filled_reset con PrePa_O_EI
    PrePa_O_EI = pd.merge(df1, diff_pivot_table_reset, on=index, how='left')    
    return PrePa_O_EI
def calc_diff_V1(df1: pd.DataFrame,df2: pd.DataFrame, index= ['NOMPROVEEDOR', 'ESTADO']) -> pd.DataFrame:
    """ entran 2 dataframes , los proceso , y en base a los df procesados 
        calculo la diferencia en una nueva columna, 
        y añado esta columna de diferencia al df actual """
    clases_unicas = pd.concat([df1['ESTADO'], df2['ESTADO']]).unique() #Creo listas de claves unicas
    df1_proces = process_df_V1(df1,clases_unicas)
    df2_proces = process_df_V1(df2,clases_unicas)
    
    df_diff = df1_proces - df2_proces
    diff_pivot_table_reset = df_diff.reset_index()
    diff_pivot_table_reset.rename(columns={'PEND_FACT_SOLES': 'DIFERENCIA EN SOLES'}, inplace=True)
    # Merge diff_pivot_table_filled_reset con PrePa_O_EI
    PrePa_O_EI = pd.merge(df1, diff_pivot_table_reset, on=index, how='left')    
    return PrePa_O_EI

def filtro(df):
    return df[df.RESPONSABLE2 == 'Eduardo Iberico'] #Filtro 
    
def get_2_df_from_path(path: str,extension: str, Week_Int: int,sheet_name: str) -> list[pd.DataFrame]:
    """ Extrae los archivos mas recientes de un path"""
    path= path+'/*'
    tipo_de_archivo = '*'+ extension
    archivos = glob.glob(path + tipo_de_archivo)
    archivos_ordenados = sorted(archivos, key=os.path.getmtime, reverse=True)
    nombres_de_archivos = [os.path.basename(archivo) for archivo in archivos_ordenados[:1] + archivos_ordenados[Week_Int-1:Week_Int]] 
    print(nombres_de_archivos)
    dataframes = []  # Lista para almacenar los DataFrames
    for nombre in nombres_de_archivos:
        path_of_e = path[:-1] + '{}'.format(nombre)
        df = pd.read_excel(path_of_e, sheet_name=sheet_name, dtype={'COMENTARIO': str})
        dataframes.append(df)
    return dataframes
        
def convert_to_date(val):
    try:
        return pd.Timestamp('1899-12-30') + pd.Timedelta(int(float(val)), 'D')
    except ValueError:
        return val

def get_recent_df(Carpeta_path: str, sheet_name: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.xlsx'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    archivo_mas_reciente = max(archivos, key=os.path.getmtime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,nombre_del_archivo_N
def get_recent_c_df(Carpeta_path: str, sheet_name: str):
    """ Devuelve el df de la hoja especifica, del archivo mas reciente de la carpeta especificada"""
    Path_n= Carpeta_path + '/*'
    tipo_de_archivo = '*.xlsx'
    # Busca el archivo más reciente
    archivos = glob.glob(Path_n + tipo_de_archivo)
    archivo_mas_reciente = max(archivos, key=os.path.getctime)
    nombre_del_archivo_N = os.path.basename(archivo_mas_reciente)
    print(archivo_mas_reciente)
    # Lee el archivo sin especificar el tipo de datos
    df = pd.read_excel(archivo_mas_reciente , sheet_name=sheet_name)
    # Si la columna "COMENTARIO" existe, cambia su tipo de datos a str
    if 'COMENTARIO' in df.columns:
        df['COMENTARIO'] = df['COMENTARIO'].astype(str)
    return df,nombre_del_archivo_N
def cleanrows(df):
    if 'STATUSFINAL' in df.columns:
        df = df[df['STATUSFINAL'] == 'PENDIENTE']
    elif 'STATUS FINAL' in df.columns:
        df = df[df['STATUS FINAL'] == 'PENDIENTE']
    return df

def process_to_bcsv(df : pd.DataFrame,ruta_del_csv : str,Fecha: str):

    df['TIME'] = Fecha
    pivot_table = pd.pivot_table(df, values='PEND_FACT_SOLES', index=['TIME','MES-COMPROMISO','NOMPROVEEDOR','ESTADO'], aggfunc=pd.Series.sum) #Agrupo 
    df_reset = pivot_table.reset_index(drop=False)
    #convetir la columna del agrupado al formato d efehca 
    # Convierte la columna 'Fecha' a datetime
    df_reset['TIME'] = pd.to_datetime(df_reset['TIME'],format = '%d-%m-%Y')
    
    # Formatea la columna 'Fecha'
    df_reset['TIME Format'] = df_reset['TIME'].apply(lambda x: format_date(x, 'EEE dd-MM-yyyy', locale=Locale('es', 'ES')))
    df_reset.to_csv(ruta_del_csv, mode='a', header=False,index=False)
def transformar_name(valor):
    str(valor)
    if valor.isupper():
        return valor
    else:
        palabras = valor.split()
        return ' '.join([palabras[0], palabras[2]])
def process_PAP(value):
    if any(map(str.isdigit, str(value))):
        value_float = int(value)
        value_str = str(value_float)
        return value_str
    else:
        return value
def load_df_by_name(directorios:str, cadena:str) -> pd.DataFrame:
    df_list = []
    for dictory in directorios:
        ruta = buscar_archivo_mas_antiguo(dictory, cadena)
        try:
            df = pd.read_excel(ruta, sheet_name='Hoja2')
            #print('Machea con nombre de hoja2')
        except Exception as e:
            print(f"Error: {e}. Trying with 'Sheet1'")
            df = pd.read_excel(ruta, sheet_name='Sheet1')
            #print('Machea con nombre de Sheet1')
        print(ruta)
        df_list.append(df)
    return df_list
def buscar_archivo_mas_antiguo(directorio, cadena):
    archivos_coincidentes = [os.path.join(directorio, nombre_archivo) for nombre_archivo in os.listdir(directorio) if cadena in nombre_archivo]
    if not archivos_coincidentes:
        return None
    else:
        archivo_mas_antiguo = min(archivos_coincidentes, key=os.path.getmtime)
        return archivo_mas_antiguo
def formated2bi(df_diff:pd.DataFrame,contrata_dic:dict,columns_dic:dict,colum_list):
    df_fomarted = df_diff.copy()
    # Aplicar la función a los nombres de las columnas
    df_fomarted.columns = map(transformar_nombre, df_fomarted.columns) # Normalizo nombres de columna
    df_fomarted['SITE'] = df_fomarted['SITE'].str.replace('_', ' ')
    df_fomarted['SITE'] = df_fomarted['SITE'].str.title()
    df_fomarted['TEXTO BREVE'] = df_fomarted['TEXTO BREVE'].str.title()
    df_fomarted['NOMPROVEEDOR'].replace(contrata_dic,inplace=True)
    df_fomarted['NOMPROVEEDOR'] = df_fomarted['NOMPROVEEDOR'].str.title()
    df_fomarted.columns = df_fomarted.columns.str.strip()
    #Cambio los nombres de las columnas con un Map
    df_fomarted.rename(columns=columns_dic, inplace=True)

    df_fomarted.columns = df_fomarted.columns.str.title()
    
    try:
        df_fomarted = df_fomarted[colum_list] #@encapsular
        print('Formato con CF')
    except:
        #colum_list.remove('Estado Cenfile')
        colum_list.remove('Antiguamiento Proyectado')
        df_fomarted = df_fomarted[colum_list]
        print('Formato sin CF')
    
    #columns_2_drop = [col.title() for col in columns_2_drop]
    
    #df_fomarted.drop(columns=columns_2_drop,inplace=True)
    return df_fomarted
def transformar_nombre(nombre):
    nombre = nombre.upper()
    nombre = nombre.replace('_', ' ')
    nombre = nombre.replace('-', ' ')
    return nombre

contrata_dic = {'DELTA ELECTRONICS (PERU) INC. S.R.L ELTEK PERU S.R.L.':'DELTA',
                'COMUNICACION FUTURA SOCIEDAD ANONIM':'COMFUTURA'}
columns_dic = {"CONTRA.FEC":"Fecha Contrata" , "CONTRAT.COM": "Comentario Contrata"
               ,"Dif Soles":"Diferencia Soles", "DOC COMPRAS": "Orden de Compra" 
               , "DOCUMENTO REFERENCIA": "DOCUMENTO REFERENCIA","NOMPROVEEDOR" :"CONTRATISTA"
               , "PAP":"N° PAP", "PEND FACT SOLES":"PENDIENTE EN SOLES"
               , "TEXTO BREVE": "DESCRIPCION"
               , "POSIC":"POSICIÓN","DESCRIPCION": "DESCRIPCIÓN"
               , "ANT PROYECTADO": "Antiguamiento Proyectado", "ANTIGUAMIENTO PAP" : "ANTIGUAMIENTO PAP"
               , "FECHA REPORTE": "FECHA REPORTE", "MES COMPROMISO" : "MES DE COMPROMISO"
               , "POS": "POSICION", 'FECH CONTAB':'FECHA CONTABILIDAD'}
def clean_date(df,column):
    df.loc[:,column] = pd.to_datetime(df[column])
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d')
    df[column] = df[column].astype(str)
    return df
# Ordeno los KEYS
def change_rows(df_concat:pd.DataFrame,df_duplicados:pd.DataFrame,Prepa_N:pd.DataFrame):
    df_concat = df_concat.sort_values(by=['CONCA', 'DOCUMENTO_REFERENCIA'], ascending=[True, True])
    # Crear un DataFrame con las keys y las columna que me interesa.
    df_info = df_duplicados.drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'], keep='first')[['CONCA', 'DOCUMENTO_REFERENCIA', 'FECHASCOMPROMISO']]
    
    #la base en la que voy a editar las columnas,borro esa columna
    df_dupli = df_duplicados.drop('FECHASCOMPROMISO', axis=1).drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'], keep='last')
    df_final = merge_dataframes(df_dupli,df_info,['CONCA','DOCUMENTO_REFERENCIA'],index=df_dupli.index) #concateno, es como si intercambiara columnas
    df_final
    return Prepa_N.update(df_final,overwrite=True)#Finalmente con eso sobreescribro el original 


def drop_duplicatesR(df_duplicados:pd.DataFrame,BDF:pd.DataFrame,Prepa_N:pd.DataFrame):
    df2Rem = df_duplicados.drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'],keep='first')
    list2Rem = df2Rem.index.tolist()
    BDF_no_D = BDF.drop(list2Rem,axis=0)
    Prepa_N = pd.concat([BDF_no_D, Prepa_N]) # este es un temporal para comprobar si hay filas duplicadas 
    print(f"los siguientes index en el BDF estan en el reporte aun: {list2Rem}")
    Prepa_N = Prepa_N.drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'])
    return Prepa_N
def merge_dataframes(df1, df2, merge_on, index):
    
    merged_df = pd.merge(df1, df2, on=merge_on, how='left')
    display(len(merged_df))
    #display('Filas duplicadas:',merged_df[merged_df.duplicated(subset=['DOCUMENTO_REFERENCIA','CONCA'])]) 
    #return merged_df
    #merged_df = merged_df.drop_duplicates()
    merged_df = merged_df.drop_duplicates(subset=['CONCA','DOCUMENTO_REFERENCIA'],keep='first')
    #return merged_df
    print("filas del merge: ",len(merged_df))
    print("filas index: ",len(index))
    merged_df.set_index(index, inplace=True)
    return merged_df
def normalize_company_names(df, column):
    """Normaliza los nombres de las empresas en la columna especificada del DataFrame."""
    # Reemplaza "SAC" o "S. A. C." al final de los nombres de las empresas con "S.A.C"
    df[column] = df[column].str.replace(r"(SAC|S\. ?A\. ?C\.)$", "S.A.C.", regex=True)
    return df
def compact_rows(df:pd.DataFrame, columns:list, delimiter:str ='/'):
    for col in df.columns:
        df[col] = df[col].astype(str)
    
    # Usar 'join' como función de agregación para concatenar los valores
    agg_func = {col: lambda x: '/'.join(x.unique()) for col in df.columns if col not in  columns}
    
    # Agrupar por 'OC Posición' y aplicar la función de agregación
    df1 = df.groupby(columns).agg(agg_func).reset_index()
    return df1
def update_and_rename(df1, update_cols, new_names):
    df = df1.copy()
    for col in update_cols:
        df[col[0]].update(df[col[1]])
    df.rename(columns=new_names, inplace=True)
    df.drop(columns=[col for sublist in update_cols for col in sublist[1:]], inplace=True)
    return df

Hoy es: 01.07


# Cargo Archivos

## Cargo Prepa_old

In [3]:
PrePa_O,_ = get_recent_c_df(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa','Sheet1')
#PrePa_O,_ = get_recent_df(r'D:\Prepa_local','Sheet1')

## Manejo los comentarios por si tiene fechas 
PrePa_O['COMENTARIO'] = PrePa_O['COMENTARIO'].apply(convert_to_date)
#CPX['FECHA'] = CPX['FECHA'].apply(convert_to_date)
PrePa_O = PrePa_O.dropna(subset=['CONCA'])



\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa\PREPA_UPDATE28.06.xlsx


## OTS

In [4]:
OT,_= get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\OTs','Hoja2')
OT = OT[OT['Status OT'] != 'PDTE CONTRATA'] #	
OT = OT[OT['Status OT'] != 'PDTE RESPONSABLE'].copy()
OT['Fecha de Creación'] = pd.to_datetime(OT['Fecha de Creación'],dayfirst=True).dt.date
OT['Fecha de Creación'] = pd.to_datetime(OT['Fecha de Creación'], format='%d-%m-%Y')
fecha_limite = pd.Timestamp('2021-11-01')
OT_cut = OT[OT['Fecha de Creación'] > fecha_limite].copy() # Corto las OTs solo de este año 
OT_cut = normalize_company_names(OT_cut,'Contrata')


D:\Scripts1\Code\ActPEA\archvis\OTs\OTS 24.06.2024.xlsx


C:\Users\C26764\AppData\Local\Temp\ipykernel_28132\3491332154.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  OT['Fecha de Creación'] = pd.to_datetime(OT['Fecha de Creación'],dayfirst=True).dt.date


## Cargo Prepa_N

In [5]:
from datetime import datetime


In [6]:
Prepa_N,nombre_del_archivo_N = get_recent_df('D:/Scripts1/Code/ActPEA/archvis/Pre_pa/NEW/','DATA')

Prepa_N = cleanrows(Prepa_N)

Fecha_reporte = nombre_del_archivo_N[-13:-8].replace('.','-')
#Prepa_N.ESTADO_CENFILE.replace(0,'No Aplica',inplace=True)
#aarchivos = glob.glob(Path_n + tipo_de_archivo)
Fecha_reporte_DT = datetime.strptime(Fecha_reporte, "%d-%m")
Today2C = datetime.strptime(Today_str[:-5], "%d-%m")
a = Today2C - Fecha_reporte_DT
if Fecha_reporte_DT == Today2C:
    reporte_old = False
    print("El reporte es actual")
elif Fecha_reporte_DT < Today2C:
    report_old = True
    print(f"El reporte es de hace {a.days} dias")


D:/Scripts1/Code/ActPEA/archvis/Pre_pa/NEW\Prepasivo completo (Finanzas Junio) 27.06.24.xlsx
El reporte es de hace 4 dias


In [7]:
with open(r"D:\Scripts1\Code\ActPEA\CODE\Temps\Standar_columns_PREPA.pickle", "rb") as archivo:
    lista_recuperada = pickle.load(archivo)
columns_E = lista_recuperada
columns_Ac = Prepa_N.columns.tolist() 

if columns_Ac != columns_E:
    Columns_diff = list(set(columns_Ac) - set(columns_E))
    respuesta = input(f'Hay una variacion de columnas en la Base,{Columns_diff} \n¿Deseas continuar con la ejecución? (s/n): ')
    if respuesta.lower() != 's':
            sys.exit()
           

Hay una variacion de columnas en la Base,['AVANCE AL 21.06', 'INDICADOR2', 'ÁREA', 'AVANCE AL 19.06', 'CODIGO_SITE', 'NOMBRE_SITE', 'PENDIENTE', 'AVANCE AL 24.06', 'STATUS FINAL', 'RESPON', 'ESTADO_CENFILE', 'CENFILE', 'AVANCE AL 17.06', 'AVANCE AL 10.06', 'AVANCE AL 27.06', 'FECHA - PROY', 'Comentario', 'AVANCE AL 26.06', 'AGRUPADOR', 'AVANCE AL 12.06'] 
¿Deseas continuar con la ejecución? (s/n):  s


## Cargo PAP

In [8]:
PAP_O,_ = get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\PAP','Hoja2')# PAP de Otros

PAP_A,_ = get_recent_df(r'D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo','Hoja2')# PAP de Admin

PAP = pd.concat([PAP_O,PAP_A])

D:\Scripts1\Code\ActPEA\archvis\PAP\PAP 01.07.xlsx
D:\Scripts1\Code\ActPEA\archvis\PAP\Administrativo\PAP_A_29.06.2024.xlsx


## Cargo los Bloqueos de Factura

In [9]:
BDF = pd.read_csv(r'D:\Scripts1\Code\ActPEA\CODE\Temps\BAF_4FC.csv')
if 'STATUSFINAL' in Prepa_N.columns:
    BDF['STATUSFINAL'] == 'PENDIENTE'
    deep_columns = BDF[['MES - COMPROMISO','STATUS FINAL','FECHASCOMPROMISO']]  

elif 'STATUS FINAL' in Prepa_N.columns:
    BDF['STATUS FINAL'] == 'PENDIENTE'
    deep_columns = BDF[['MES - COMPROMISO','STATUS FINAL','FECHASCOMPROMISO']]  
else:
    deep_columns = BDF[['MES - COMPROMISO','FECHASCOMPROMISO']]  

BDF.drop_duplicates(subset=['CONCA','DOCUMENTO_REFERENCIA'],inplace=True)
BDF.drop([16],axis=0,inplace=True) # duplicado
# Selecciona las primeras 10 columnas
first_ten = BDF.iloc[:, :8]

# Selecciona las columnas 49 y 76

# Combina las columnas seleccionadas en un nuevo DataFrame
BDF_cut = pd.concat([first_ten, deep_columns], axis=1)

In [10]:
BDF_cut.dropna(subset='CONCA',inplace=True)
BDF_cut.CONCA = BDF_cut.CONCA.astype('int64')
BDF_cut.rename(columns={'FECHASCOMPROMISO':'FECHAS COMPROMISO'},inplace=True)
BDF_cut.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112 entries, 0 to 112
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   DOC_COMPRAS           112 non-null    float64
 1   POSIC                 112 non-null    float64
 2   CONCA                 112 non-null    int64  
 3   DOCUMENTO_REFERENCIA  112 non-null    float64
 4   TEXTO_BREVE           112 non-null    object 
 5   PEND_FACT_SOLES       112 non-null    float64
 6   NOMPROVEEDOR          112 non-null    object 
 7   RESPONSABLE2          112 non-null    object 
 8   MES - COMPROMISO      112 non-null    object 
 9   STATUS FINAL          112 non-null    object 
 10  FECHAS COMPROMISO     112 non-null    object 
dtypes: float64(4), int64(1), object(6)
memory usage: 10.5+ KB


## Compruebo las filas que voy a insertar
- La variable reporte_old(boolean) indica si el reporte de prepa es nuevo o antiguo
- La variable row_e indica si las filas del bloque existen en el PREPA_N

### Para crear la variable de filas duplicadas 

In [11]:
# Concatenar los dos DataFrames
df_concat_0 = pd.concat([BDF_cut, Prepa_N]) # este es un temporal para comprobar si hay filas duplicadas 
# Crear una copia del DataFrame concatenado
df_copia = df_concat_0.copy()
# Eliminar las filas duplicadas basándose en las columnas de identificación
df_final_0 = df_concat_0.drop_duplicates(subset=['CONCA', 'DOCUMENTO_REFERENCIA'], keep='last')
# Obtener las filas duplicadas
df_duplicados = df_copia[df_copia.duplicated(subset=['CONCA', 'DOCUMENTO_REFERENCIA'], keep=False)] ## Estas son las duplicas, 
#si este no es vacio entonces mi variable True Para duplicados
row_e = not(df_duplicados.empty)
df_duplicados.PEND_FACT_SOLES.sum()

0.0

In [12]:
if (row_e == True) and (report_old==True):
    print("Las filas existen en el Prepa_N, solo cambio datos")
    Prepa_N = change_rows(df_concat_0,df_duplicados,Prepa_N)
elif (row_e == True) and (report_old==False):
    Prepa_N = drop_duplicatesR(df_duplicados,BDF_cut,Prepa_N)
else: 
    print("Las filas no existen en el reporte, concateno")
    Prepa_N = pd.concat([Prepa_N, BDF_cut]) 
    Prepa_N.reset_index(drop=True,inplace=True)


Las filas no existen en el reporte, concateno


In [13]:
Prepa_N['MES - COMPROMISO'].value_counts()

MES - COMPROMISO
Julio            230
Bloqueo de AF    112
Agosto            98
Setiembre         55
Junio             24
Mayo              23
Octubre            8
Name: count, dtype: int64

# Prepocesing

## Preprosecing of OTs

In [14]:
OT_cut.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2957 entries, 95 to 3787
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   OT                    2957 non-null   int64         
 1   Origen                2957 non-null   object        
 2   Fecha de Creación     2957 non-null   datetime64[ns]
 3   N. Req                2957 non-null   int64         
 4   Codigo de Site        2957 non-null   object        
 5   Nombre de Site        2957 non-null   object        
 6   Región                2957 non-null   object        
 7   Tipo Req              2957 non-null   object        
 8   Proyecto              2957 non-null   object        
 9   Subproyecto           2957 non-null   object        
 10  Etiqueta              209 non-null    object        
 11  Prioridad             2955 non-null   float64       
 12  Solicitante           2957 non-null   object        
 13  Gerente Construcción  

In [15]:
Proyectos = ['ROLLOUT - 2023','ROLLOUT - 2022','ROLLOUT - 2024','CANON - 2023',]
OT_cut_mant = OT_cut[OT_cut['Tipo Req'] == 'MANT. MEJORA DE RED'].copy()
OT_cut_M = OT_cut[OT_cut.Proyecto == 'EXPANSIÓN'].copy()
OT_cut_RRL = OT_cut[OT_cut.Etiqueta.isin(Proyectos)].copy()

OT_concat = pd.concat([OT_cut_M,OT_cut_RRL,OT_cut_mant],axis=0)
OT_concat.rename(columns={'Codigo de Site':'ID_SITIO',
                           'Contrata':'NOMPROVEEDOR',
                           'Nombre de Site':'SITE'},inplace=True)
OT_concat = OT_concat[['OT','ID_SITIO','SITE','Proyecto','NOMPROVEEDOR','Etiqueta','Status OT']].copy()

OT_agg_ID_PRO = compact_rows(OT_concat,['SITE','NOMPROVEEDOR'],'/')
#OT_agg_ID_NM_PRO = compact_rows(OT_cut_RRL_2M,['ID_SITIO','SITE','PROVEEDOR'],'/')

## Preprosecin of PAP

In [16]:
PAP_f = PAP[['OC Posición','N° Sol','Estado','Acción','Nombre responsable','Id.SIte',
             'SIte','# Días','F.Creación','F.Modifica']]
##Filtro solo del norte
PAP_f = PAP_f.dropna(subset=['Id.SIte'])
PAP_f = PAP_f[PAP_f['# Días'] < 500]

PAP_f_N = PAP_f[PAP_f['Id.SIte'].str.startswith(('L','T','SAD','CL','CAC'))].copy()
PAP_f_N = PAP_f_N.rename(columns={'N° Sol': 'PAP', #Rename
                               'SIte': 'SITE',
                               'Estado': 'ESTADO_PAP',
                                '# Días': 'ANTIGUAMIENTO_PAP',
                                'Id.SIte': 'ID Site',
                             'Nombre responsable': 'RESPONSABLE_PAP' })
PAP_S = Ac.split_ocs(PAP_f_N) # Spliteo OCs


## Preprosecing of PREP

In [17]:
def pre_proces(df: pd.DataFrame ,columns_2str: list[str] ,column_filter: str,C_format) -> pd.DataFrame: 
    """Anotaciones del tipo de cada parametro para hacer la funcin mas descriptiva"""
    ## Preprosecing of PREP_NEW
    df = convert_columns(df.copy(),columns_2str,C_format ) #Convert to str a key column
    df_EI = df.loc[df[column_filter] == 'Eduardo Iberico']#Filter
    df_EI = df_EI.copy()  # Crea una copia del DataFrame original para evitar modificar los datos originales
    return df_EI
def convert_columns(df, columns,type):
    for column in columns:
        df[column] = df[column].astype(type)
    return df

In [18]:
############################################### PASO 01#####################################################################
Prepa_EI_NC = pre_proces(Prepa_N,['DOC_COMPRAS', 'POSIC','CONCA'],'RESPONSABLE2','int64')
#####################################################################################################
###############################################################################################3
Prepa_EI = pre_proces(Prepa_EI_NC,['DOC_COMPRAS', 'POSIC','CONCA'],'RESPONSABLE2','str')
                    
Prepa_EI.loc[:, "OC Posición"] = Prepa_EI.loc[:, "DOC_COMPRAS"].str.cat(Prepa_EI.loc[:, "POSIC"], sep= ":")
Monto_In = Prepa_EI["PEND_FACT_SOLES"].sum() #Mont of USD 
#Prepa_EI = Prepa_EI.drop(columns=['FECHAS COMPROMISO','MES - COMPROMISO'])
#Prepa_EI.DOCUMENTO_REFERENCIA = Prepa_EI.DOCUMENTO_REFERENCIA.astype(str)

In [19]:
len(Prepa_EI)

179

In [20]:
## Preprosecing of PREP_OLD
PrePa_O_EI = pre_proces(PrePa_O,['DOC_COMPRAS', 'POSIC','CONCA'],'RESPONSABLE2','str')
PrePa_O_EI.replace('0x2a', np.nan, inplace=True)
PrePa_O_EI['PAP'] = PrePa_O_EI['PAP'].replace('sin pap', np.nan)
PrePa_O_EI = PrePa_O_EI.rename(columns={'FECHA': 'F.Creación'}) #Rename
#Filter of PEA_old
PrePa_O_EI = PrePa_O_EI[['DOCUMENTO_REFERENCIA','PAP','CONCA', 'SITE', 'COMENTARIO','ESTADO', 'RESPONSABLE3','CONTRA.FEC','CONTRAT.COM']] #Columns i need 4 heredar
PrePa_O_EI.CONCA = PrePa_O_EI.CONCA.astype('int64')
Prepa_EI.CONCA = Prepa_EI.CONCA.astype('int64')
Prepa_EI = Prepa_EI.drop_duplicates(subset=['DOCUMENTO_REFERENCIA','CONCA'],keep='first')
len(PrePa_O_EI)

179

# Paso 02 (heredo info)

In [21]:
## ########################### # Merge new con el old (PASO 02) ####################################################################################
#####################################################################################################################3
PrePa_act = merge_dataframes(Prepa_EI,PrePa_O_EI,['CONCA','DOCUMENTO_REFERENCIA'], index=Prepa_EI.index)### Info heredada del old REDY 

#Mantengo el index para luego
###### Creo columnas para el que actualice el PAP####################### ///
PrePa_act[['ANTIGUAMIENTO_PAP','F.Creación','Acción','RESPONSABLE_PAP','F.Modifica']] = ''
PrePa_act.rename(columns={'ESTADO' : 'ESTADO_PAP'},inplace=True)
PrePa_act['RESPONSABLE_PAP'] =PrePa_act['RESPONSABLE_PAP'].astype(str)
Cash_in = PrePa_act.PEND_FACT_SOLES.sum() 


179

filas del merge:  179
filas index:  179


# Paso 03

In [22]:
PRE_all_act =  Ac.update_values_optimized_V2(PrePa_act, PAP_S, "OC Posición",['PAP','SITE','ESTADO_PAP','Acción','F.Creación','F.Modifica',##añadir una columna nueva
                                            'RESPONSABLE_PAP','ANTIGUAMIENTO_PAP']) 
#PRE_all_act.drop(columns = 'OC Posición',inplace=True)
display(PRE_all_act.PEND_FACT_SOLES.sum())


duplicated_index = PRE_all_act.index.duplicated(keep=False)
df_duplicated = PRE_all_act[duplicated_index].copy()
PRE_all_act.drop(PRE_all_act[duplicated_index].index, inplace=True) # Borro duplicados del principal para tratarlos en el otro datafra 



# Ordena el DataFrame por la columna de fechas  LA FECHA DE CREACION ME SIRVE PARA ELMINAR DUPLICADOS
df_duplicated['F.Modifica'] = pd.to_datetime(df_duplicated['F.Modifica'], format="%d/%m/%Y %I:%M:%S %p") # Paso a fecha para ordenar
df_duplicated.sort_values('F.Modifica',ascending=False, inplace=True)
df_duplicated_NR = df_duplicated[df_duplicated.ESTADO_PAP != 'Rechazado'].sort_index(ascending=False)

df_duplicated_NR = df_duplicated_NR.loc[~df_duplicated_NR.index.duplicated(keep='first')] # niego la condicional que me selecciona cada indice duplicado excepto el primero
#df_duplicated['F.Creación'] = df_duplicated['F.Creación'].dt.strftime("%m/%d/%Y %I:%M:%S %p") # Regreso a str 
PRE_all_act = pd.concat([PRE_all_act, df_duplicated_NR])
PRE_all_act = cleanrows(PRE_all_act)

#PRE_all_act = PRE_all_act.drop(columns=['F.Modifica'])
Cash_out = PRE_all_act.PEND_FACT_SOLES.sum()
print(Cash_in - Cash_out)
#PRE_all_act['ESTADO_PAP'] = PRE_all_act['ESTADO_PAP'].fillna('Finalizado')
PRE_all_act.ESTADO_PAP.value_counts(dropna=False)

2314466.2

0.0


ESTADO_PAP
Pre registro        53
Finalizado          39
Visita ejecutada    20
Aprobaciones FAC    17
FAC                 17
Observado           11
Registrado           8
Aprobaciones PAC     6
PAC                  5
Programado           2
Sin pap              1
Name: count, dtype: int64

In [23]:
PRE_all_act['RESPONSABLE_PAP'] = PRE_all_act['RESPONSABLE_PAP'].replace('','SIN RESPONSABLE')
# Aplicar la transformación
PRE_all_act['RESPONSABLE_PAP'] = PRE_all_act['RESPONSABLE_PAP'].apply(transformar_name)

### Mapeo Compuesto (Tratamiento de Estados)

In [24]:
# Definimos los valores que estamos buscando en una lista
valores_buscados = ['Registrado', 'Aprobaciones FAC', 'Aprobaciones PAC','Lista Pendientes']

# Creamos la nueva columna basada en la condición
PRE_all_act['ESTADO'] = np.where(PRE_all_act['ESTADO_PAP'].isin(valores_buscados), 
                                 PRE_all_act['RESPONSABLE_PAP'], 
                                 PRE_all_act['ESTADO_PAP'])
map_estado = {'Auto ATP' : 'Registrar lista de pendientes',
       'Visita ejecutada' : 'Registrar lista de pendientes',
        'Programado' : 'WILBERT ATP',
        'FAC' : 'Solic FAC',
        'Gustavo Chocos':'CHOCOS',
        'Edwin Hurtado':'Edwin',
        'Marco Mejia':'Marco',
        'Wilbert Pintado':'Wilbert',
        'Orlando Zapata':'ORLANDO Z.',
        'Miguel Rojas':'MIGEL ROJAS',
        'Julio Arciniega':'JULIO ARCENIEGA',
        'Juan Trigo':'JUAN JOSE'}
PRE_all_act['ESTADO'] = PRE_all_act['ESTADO'].replace(map_estado)
#Modifico el valor par adetemrina condicion
PRE_all_act.loc[PRE_all_act['ESTADO'] == 'SIN RESPONSABLE', 'ESTADO'] = 'Por Asignar'

a = PRE_all_act.ESTADO.value_counts(dropna=False).reset_index()
a.to_csv(f'D:/Prepa_R/Recuento_estados-{Today_D_M}.csv',index_label=False)
display(a)

,ESTADO,count
0,Pre registro,53
1,Finalizado,39
2,Registrar lista de pendientes,20
3,Solic FAC,17
4,Edwin,14
5,Observado,11
6,Eduardo Iberico,6
7,CHOCOS,5
8,PAC,5
9,Por Asignar,3


In [25]:
PRE_all_act = PRE_all_act.drop(columns=['F.Creación','RESPONSABLE_PAP'
                                        ,'ANTIGUAMIENTO','DEMORA','FECH_LIB_AF','INDICADOR','MON'])


In [26]:
display(PRE_all_act[PRE_all_act.ESTADO.isna()])


,DOC_COMPRAS,POSIC,CONCA,DOCUMENTO_REFERENCIA,FONDO,TIPO_MAT,TEXTO_BREVE,CODPROVEEDOR,NOMPROVEEDOR,FECH_CONTAB,...,PAP,SITE,COMENTARIO,ESTADO_PAP,RESPONSABLE3,CONTRA.FEC,CONTRAT.COM,Acción,F.Modifica,ESTADO


In [27]:
len(PRE_all_act)

179

In [28]:
import pandas as pd

# Supongamos que df es tu DataFrame
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# Enbullo la info actualizada del norte en el global
    04. Para ello necesito primero darle formato al global

In [29]:
Resultado = PRE_all_act
columnas_para_actualizar = ['ANTIGUAMIENTO_PAP', 'PAP','SITE','COMENTARIO','RESPONSABLE3','CONTRA.FEC','CONTRAT.COM','ESTADO_PAP','ESTADO']
Resultado_reducido = Resultado[columnas_para_actualizar]
#############################################################
Prepa_N = Prepa_N.dropna(axis=1, how='all')
unhavent_columns = set(Resultado.columns.tolist()) - set(Prepa_N.columns.tolist()) # Columnas que le faltan al grande del chico
# Crear las columnas necesarias con dtype 'object'
for col in list(unhavent_columns):
    Prepa_N[col] = pd.Series(dtype='object')
## Df pequeño en el Df grande 
Resultado.columns = Resultado.columns.str.replace(' ', '')
Prepa_N.columns = Prepa_N.columns.str.replace(' ', '')
Prepa_N.update(Resultado_reducido, overwrite=True) ### Aqui es dond eembullo la data.
Prepa_N.columns = Prepa_N.columns.astype(str)
Prepa_N_1 = Ac.convert_columns_to_str(Prepa_N.copy(), ['COMENTARIO'])
Prepa_N_1 = Prepa_N_1.drop(columns=['ESTADO_PAP','Acción','SOLICITUD','ESTATUS_SOLICITUD','AREA_RESPONSABLE','ACCION','TIPO_MAT','CE_GESTOR'])
Prepa_N_1 = Prepa_N_1.replace({'nan':''})
cols = list(Prepa_N_1.columns)
Prepa_N_1 = Prepa_N_1[cols[:-9] + ['ANTIGUAMIENTO_PAP','PAP', 'SITE', 'ESTADO','COMENTARIO','RESPONSABLE3','CONTRA.FEC','CONTRAT.COM']]
Prepa_N_1.COMENTARIO = Prepa_N_1.COMENTARIO.replace({'1899-12-30 00:00:00' : ''})
Prepa_N_1 = Prepa_N_1.dropna(axis=1, how='all') # elimino columnas vacias
Prepa_N_1 = Prepa_N_1.loc[:,~Prepa_N_1.columns.duplicated()]

# to Excel
 #### 1. 4all
 #### 2. 4me

In [30]:
import datetime


In [31]:
#
useless_columns= ['DOCUMENTO_REFERENCIA','FONDO','FECH_CONTAB','MON','INDICADOR',
                      'PEND_FACT','EA_EM','ESTATUS_ACEPT','RESPONSABLE','DEMORA',
                  'SUB_DIRECCION','FECHA_REPORTE','ELEMENTO_PEP','CENFILE','ESTADO_CENFILE','CODIGO_SITE','INDICADOR2',
                      'ÁREA','AGRUPADOR','RESPON','NOMBRE SITE','RESPONSABLE_PAP']
## Formato Tabla en el excel 
#Today_D_M=Today_D_M+'_2'
Ac.Excel_format(Prepa_N_1,r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa\PREPA_UPDATE{}.xlsx'.format(Today_D_M),useless_columns)
Ac.Excel_format(Prepa_N_1,r'D:\Prepa_local\PREPA_UPDATE{}.xlsx'.format(Today_D_M),useless_columns)

filename = r'D:\Scripts1\Code\ActPEA\CODE\last_run.json'
# Carga la última fecha de ejecución
last_run_date = Ac.load_last_run_date(filename)

# Comprueba si la celda ya se ha ejecutado hoy
if last_run_date != datetime.datetime.now().date():
    # Tu código aquí
    print('Ejecutado: ',Today_str)
    #process_to_bcsv(PRE_all_act,'D:/Prepa/TIME.S/Prepa_TS1.csv',Today_str)
    process_to_bcsv(PRE_all_act,r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Prepa_TS.csv',Today_str)# añado al acumulado

    # Guarda la fecha de hoy como la última fecha de ejecución
    Ac.save_last_run_date(filename)
else:
    print("El código ya se ha ejecutado hoy.")


Ejecutado:  01-07-2024


# To BI

In [32]:
def formated2bi(df_diff:pd.DataFrame,contrata_dic:dict,columns_dic:dict,colum_list):
    df_fomarted = df_diff.copy()
    # Aplicar la función a los nombres de las columnas
    df_fomarted.columns = map(transformar_nombre, df_fomarted.columns) # Normalizo nombres de columna
    df_fomarted['SITE'] = df_fomarted['SITE'].str.replace('_', ' ')
    df_fomarted['SITE'] = df_fomarted['SITE'].str.title()
    df_fomarted['TEXTO BREVE'] = df_fomarted['TEXTO BREVE'].str.title()
    df_fomarted['NOMPROVEEDOR'].replace(contrata_dic,inplace=True)
    df_fomarted['NOMPROVEEDOR'] = df_fomarted['NOMPROVEEDOR'].str.title()
    df_fomarted['MES COMPROMISO'].replace('Setiembre','Septiembre',inplace=True)
    df_fomarted.columns = df_fomarted.columns.str.strip()
    #Cambio los nombres de las columnas con un Map
    df_fomarted.rename(columns=columns_dic, inplace=True)

    df_fomarted.columns = df_fomarted.columns.str.title()
    
    try:
        df_fomarted = df_fomarted[colum_list] #@encapsular
        print('Formato con CF')
    except:
        #colum_list.remove('Estado Cenfile')
        #colum_list.remove('Antiguamiento Proyectado')
        colum_list.remove('Estado Cenfile')
        df_fomarted = df_fomarted[colum_list]
        print('Formato sin CF')
    
    #columns_2_drop = [col.title() for col in columns_2_drop]
    
    #df_fomarted.drop(columns=columns_2_drop,inplace=True)
    return df_fomarted

In [33]:
## Revisar esto porque se ejecuto 2 veces el mismo dia pero en diferentes idles
dataframes = get_2_df_from_path(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\Resultados_prepa','.xlsx',4,'Sheet1')
#dataframes = get_2_df_from_path(r'D:\Prepa_local','.xlsx',4,'Sheet1')

# Ruta de donde extraigo los archivos a comparar,intervalo,Nombre de la hoja
dataframes_filt = list(map(filtro,dataframes)) # filtro solo del norte
df_diff = calc_diff_V1(dataframes_filt[0],dataframes_filt[1]) # Funcion que me crea la columna de diferencia.
df_diff = cleanrows(df_diff)
#display(df_diff.info()) 

df_diff['DIFERENCIA EN SOLES'] = df_diff['DIFERENCIA EN SOLES'].transform(lambda x: x / df_diff['DIFERENCIA EN SOLES'].value_counts()[x] if pd.notnull(x) else x)
#display(df_diff.info())
#columns_2_drop = ["CODPROVEEDOR","EA EM", "MON","PEND FACT", "CONCA"
                         # ,"ESTATUS ACEPT", "ELEMENTO PEP", "SUB DIRECCION","FONDO","INDICADOR", "RESPONSABLE",
                     # "DÍAS 2","FECHA CONTRATA",'Responsable Pap','Codigo Site','Nombre Site','Cenfile']
with open(r'D:\Scripts1\Code\ActPEA\CODE\Temps\ListaColumnsBI.pkl', "rb") as archivo:
    column_list = pickle.load(archivo)
df_diff = normalize_company_names(df_diff,'NOMPROVEEDOR')
df_diff_m = pd.merge(df_diff,OT_agg_ID_PRO[['SITE','NOMPROVEEDOR','Status OT']],on=['SITE','NOMPROVEEDOR'],how='left')
df_diff_m.drop(columns=['ANT-PROYECTADO'],inplace=True)
df_diff_m.rename(columns={'Status OT':'ANTIGUAMIENTO PROYECTADO'},inplace = True)


df_fomarted = formated2bi(df_diff_m,contrata_dic,columns_dic,column_list) 
#columns_E = [s.upper() for s in lista_recuperada]

['PREPA_UPDATE01.07.xlsx', 'PREPA_UPDATE27.06.xlsx']
Formato con CF


In [34]:
df_diff_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   DOC_COMPRAS               179 non-null    int64         
 1   POSIC                     179 non-null    int64         
 2   CONCA                     179 non-null    int64         
 3   DOCUMENTO_REFERENCIA      179 non-null    int64         
 4   FONDO                     67 non-null     object        
 5   TEXTO_BREVE               179 non-null    object        
 6   CODPROVEEDOR              67 non-null     float64       
 7   NOMPROVEEDOR              179 non-null    object        
 8   FECH_CONTAB               67 non-null     datetime64[ns]
 9   MON                       67 non-null     object        
 10  INDICADOR                 67 non-null     object        
 11  EA_EM                     67 non-null     float64       
 12  PEND_FACT             

In [35]:
df_fomarted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Orden De Compra           179 non-null    int64         
 1   Posición                  179 non-null    int64         
 2   Documento Referencia      179 non-null    int64         
 3   Descripcion               179 non-null    object        
 4   Contratista               179 non-null    object        
 5   Fecha Contabilidad        67 non-null     datetime64[ns]
 6   Pendiente En Soles        179 non-null    float64       
 7   Demora                    67 non-null     float64       
 8   Antiguamiento             67 non-null     object        
 9   Responsable2              179 non-null    object        
 10  Fecha Reporte             67 non-null     datetime64[ns]
 11  Antiguamiento Pap         178 non-null    object        
 12  Fechascompromiso      

## Añado estado de proyecto por coincidencia de nombre y contrata 

In [36]:
if 'Estado Cenfile' not in df_fomarted.columns:
    df_fomarted['Estado Cenfile'] = '0'
else: pass

In [37]:

df_fomarted.to_csv(r'\\LIMBIPBICOV01.claro.pe\Red Región Norte\PowerBI\Reporte Prepasivo.csv',index=False)
print('Bi actualizado')

#df_fomarted.to_csv(r'C:\Users\C26764\America Movil Peru S.A.C\EAS - 2\PBI Dashboard\Reporte Prepasivo.csv',index=False)
df_fomarted['Diferencia En Soles'].sum()  

Bi actualizado


57091.359999999964

In [38]:
df_fomarted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Orden De Compra           179 non-null    int64         
 1   Posición                  179 non-null    int64         
 2   Documento Referencia      179 non-null    int64         
 3   Descripcion               179 non-null    object        
 4   Contratista               179 non-null    object        
 5   Fecha Contabilidad        67 non-null     datetime64[ns]
 6   Pendiente En Soles        179 non-null    float64       
 7   Demora                    67 non-null     float64       
 8   Antiguamiento             67 non-null     object        
 9   Responsable2              179 non-null    object        
 10  Fecha Reporte             67 non-null     datetime64[ns]
 11  Antiguamiento Pap         178 non-null    object        
 12  Fechascompromiso      

In [39]:
print(df_fomarted['Mes De Compromiso'].value_counts())

Mes De Compromiso
Bloqueo de AF    112
Agosto            37
Julio             18
Septiembre        12
Name: count, dtype: int64
